In [327]:
# import the necessary packages
from imutils import paths
import cv2
import os
import io
import re
import base64
from PIL import Image
import selenium
import requests
import time
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [328]:
# Download more than 500 images of any given parameter
def download_extended_urls(param):
    google_url = "https://www.google.com/imghp"
    driver = webdriver.Chrome()
    try:
        driver.get(google_url)
        #browser.visit(google_url)
        que=driver.find_element_by_xpath("//input[@name='q']")
        que.send_keys(param)
        que.send_keys(Keys.RETURN)       
        element = driver.find_element_by_tag_name("body")
        # Scroll down
        for i in range(60):
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)
        try:
            driver.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div/div[5]/input').click()
            for i in range(100):
                element.send_keys(Keys.PAGE_DOWN)
                time.sleep(0.3)  # bot id protection
        except:
            for i in range(30):
                element.send_keys(Keys.PAGE_DOWN)
                time.sleep(0.3)  # bot id protection

        print("Reached end of Page.")
        time.sleep(0.5)

        source = driver.page_source #page source
        #close the browser
        driver.close()
    except Error as e:
        print(e)
        
    url_list = []
    soup = bs(source, "html.parser")
    img = soup.find_all("img", class_='rg_i Q4LuWd tx8vtf')
    for i in img:
        try:
            url_list.append(i['src'])
        except:
            pass
    with open(f'urls_for_training/{param}.txt', 'w') as filehandle:
        for listitem in url_list:
            filehandle.write('%s\n' % listitem)
    return url_list


In [329]:
def download_img_from_url(param):
    # grab the list of URLs then initialize the
    # total number of images downloaded thus far
    total = 0
    # loop the URLs
    for url in download_extended_urls(param)[:70]:
        try:
            r = requests.get(url, timeout=60)
            result_file = 'result_file'

            with open(result_file, 'wb') as file_handler:
                file_handler.write(r.content)
            try:
                os.mkdir(f"image/{param}/")
            except:
                pass
            p = os.path.join("image", param, "{}.jpg".format(str(total).zfill(8)))
            Image.open(result_file).save(p)
            os.remove(result_file)

            # update the counter
            print("[INFO] downloaded: {}".format(p))
        except:
            try:       
                z = re.findall(f'.+base64,(.+)',url)[0]
                p = os.path.join("image", param, "{}.jpg".format(str(total).zfill(8)))
                im = Image.open(io.BytesIO(base64.b64decode(z))).save(p)
            # handle if any exceptions are thrown during the download process
            except:
                print("[INFO] error downloading {}...skipping".format(p))
        total += 1
    # loop over the image paths we just downloaded
    for imagePath in paths.list_images(os.path.join("image", param)):
    # initialize if the image should be deleted or not
        delete = False

        # try to load the image
        try:
            image = cv2.imread(imagePath)
            # if the image is `None` then we could not properly load it
            # from disk, so delete it
            if image is None:
                print("None")
                delete = True
        # if OpenCV cannot load the image then the image is likely
        # corrupt so we should delete it
        except:
            print("Except")
            delete = True

        # check to see if the image should be deleted
        if delete:
            print("[INFO] deleting {}".format(imagePath))
            os.remove(imagePath)
    return

In [330]:
param='television'
download_img_from_url(param)

Reached end of Page.
[INFO] downloaded: image/television/00000020.jpg
[INFO] downloaded: image/television/00000021.jpg
[INFO] downloaded: image/television/00000022.jpg
[INFO] downloaded: image/television/00000023.jpg
[INFO] downloaded: image/television/00000024.jpg
[INFO] downloaded: image/television/00000025.jpg
[INFO] downloaded: image/television/00000026.jpg
[INFO] downloaded: image/television/00000027.jpg
[INFO] downloaded: image/television/00000028.jpg
[INFO] downloaded: image/television/00000029.jpg
[INFO] downloaded: image/television/00000030.jpg
[INFO] downloaded: image/television/00000031.jpg
[INFO] downloaded: image/television/00000032.jpg
[INFO] downloaded: image/television/00000033.jpg
[INFO] downloaded: image/television/00000034.jpg
[INFO] downloaded: image/television/00000035.jpg
[INFO] error downloading image/television/00000036.jpg...skipping
[INFO] downloaded: image/television/00000037.jpg
[INFO] downloaded: image/television/00000038.jpg
[INFO] downloaded: image/televi